# Transcriptomic Clustering

In [2]:
import transcriptomic_clustering as tc
import scanpy as sc

## Loading Data
transcriptomic_clustering functions operate on Annotation Data, from part of scanpy/anndata. This supports both dense np.ndarray's and sparse scipy.csr arrays for the matrix, and additional .['obs'], .['obsm'] and .['var'], [.varm] fields for storing information about the observations (cells) and variables (genes). Data is stored as an H5 file, allowing for both in-memory and file-backed operations. For more information, see https://anndata.readthedocs.io/en/latest/

In [3]:
# Data is load into memory. Add 'r' or 'r+' for reading and modifying filebacked annotation data
tasic_adata = sc.read_h5ad('./data/tasic2016counts_csr.h5ad')

## General API patterns
### Appending to AnnData objects
Many functions in the transcriptomic package calculate features of the data. By default, they will return the values, e.g.
``` 
means, variances = tc.get_gene_means_variances(adata)
```
However, some functions allow you to add the data to the AnnData object instead
```
tc.get_gene_means_variances(adata, annotate=True)
print(adata.var['mean_exp'])
print(adata.var['var_exp'])
```
### Modifying AnnData in Place or Making a Copy
A function that modifies AnnData will by default modify it in place.
You can make a copy by passing one of these two optional arguments
to functions that modify AnnData.X

#### In Place
Simply call the function on the AnnData object, and it will be modified in place.
```
tc.normalize(tasic_adata)
```
All functions will return a reference to the modified AnnData object, copied or not.
For example, the following is the same as the above:
```
normalized_data = tc.normalize(tasic_adata)
```
Since this is in-place, normalized_data and tasic_adata is the same object.
You can verify this, as `normalized_adata is tasic_adata` returns True.

#### In Memory
To make a copy in memory, pass `inplace=False`
For example:
```
normalized_adata = tc.normalize(tasic_adata, inplace=False)
```
In this case, normalized_adata is a copy, and `normalized_adata is tasic_adata` returns False
#### Filebacked
To make a file-backed copy, pass `copy_to=path/to/new/file.h5ad`
For example:
```
normalized_adata = tc.normalize(tasic_adata, copy_to='path/to/new/file.h5ad')
```


## Running the Pipeline

### Normalization
(insert details here)

In [4]:
normalized_adata = tc.normalize(tasic_adata)

### Highly Variable Genes

We extract highly variable genes (HVGs) to further reduce the dimensionality of the dataset and include only the most informative genes. Genes that vary substantially across the dataset are informative of the underlying biological variation in the data. As we only want to capture biological variation in these genes, we select highly variable genes after normalization. HVGs will be used for the following dimensionality reduction and clustering.

Original hicat R codes use log10 of means and dispersion for loess fit. We change it to natural log for loess fit. 

<table><tr>
<td> <img width="480" alt="hvgs" src="https://raw.githubusercontent.com/gnayuy/ai_work_related/main/loess_fit_tasic2016.png"> </td>
<td> <img width="480" alt="hvgs" src="https://raw.githubusercontent.com/gnayuy/ai_work_related/main/hvgs_tasic2016.png"> </td>
</tr></table>

Similar to normalization, we can compute means, variances, and gene_mask 

In [5]:
means, variances, gene_mask = tc.means_vars_genes(adata=normalized_adata, chunk_size=300)
tc.highly_variable_genes(adata=normalized_adata, means=means, variances=variances, gene_mask=gene_mask, max_genes=3000)
normalized_adata.var_names[normalized_adata.var['highly_variable']]

Index(['0610007N19Rik', '0610039K10Rik', '0610040J01Rik', '1110006O24Rik',
       '1110015O18Rik', '1110050K14Rik', '1190002F15Rik', '1190005I06Rik',
       '1500010J02Rik', '1500015L24Rik',
       ...
       'mt_AK131586', 'mt_AK139026', 'mt_AK141881', 'mt_AK143357',
       'mt_AK157367', 'mt_AK159262', 'mt_AK166684', 'mt_BC006023',
       'mt_BC104337', 'mt_GU332589'],
      dtype='object', length=3000)

### PCA analysis
The `tc.pca()` function uses scipy's pca solver or SKlearn under the hood

In [ ]:
(components, explained_variance_ratio, explained_variance) = \
    tc.pca(normalized_adata, cell_mask=1000, use_highly_variable=True)

### Louvain Clustering

### Future APIs
filter known modes
wcgna
ward clustering
merging
hierarchical sorting

### Managing Memory
setting limits
writing files
chunking

## Accuracy Compared to R

### Normalize
(plots/stats)

### HVGs

<table><tr>
<td> <img width="480" alt="hvgs_unchunk" src="https://raw.githubusercontent.com/gnayuy/ai_work_related/main/filtergenes_hvg_large_unchunk.png"> </td>
<td> <img width="480" alt="hvgs_chunked" src="https://raw.githubusercontent.com/gnayuy/ai_work_related/main/filtergenes_hvg_large_chunked.png"> </td>
</tr></table>

### PCA